In [ ]:
from flask import Flask, jsonify, request
import pandas as pd
from transformers import pipeline
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


# Initialize Flask app
app = Flask(__name__)

data = pd.read_csv("sales_performance_data.csv") 

# Initialize Hugging Face GPT 
model = pipeline('text-generation', model='gpt2')  


# 1. Individual Sales Representative Performance Analysis

@app.route('/api/rep_performance', methods=['GET'])
def rep_performance():
    rep_id = request.args.get('rep_id')
    
    # Find the sales representative data
    rep_data = data[data['employee_id'] == int(rep_id)]
    if rep_data.empty:
        return jsonify({"error": "Representative not found"}), 404
        
    employee_name = rep_data['employee_name'].iloc[0]  # Get the name of the representative 
    # Perform analysis based on the data
    total_leads = int(rep_data['lead_taken'].sum()) 
    total_revenue = float(rep_data['revenue_confirmed'].sum())  
    
    # Generate performance feedback using GPT 
    feedback_input = f"{employee_name} has handled {total_leads} leads and generated ${total_revenue} in confirmed revenue. Provide feedback on this performance."
    feedback = model(feedback_input, max_length=100, num_return_sequences=1, truncation=True)[0]['generated_text']
    
    return jsonify({
        "rep_id": rep_id,
        "employee_name": employee_name,
        "total_leads": total_leads,
        "total_revenue": total_revenue,
        "feedback": feedback
    })

# 2. Overall Sales Team Performance Summary

@app.route('/api/team_performance', methods=['GET'])
def team_performance():
    # Calculate overall team statistics
    total_leads = int(data['lead_taken'].sum())  
    total_revenue = float(data['revenue_confirmed'].sum()) 
    total_tours = int(data['tours_booked'].sum()) 
    
    # Generate feedback on the team's overall performance
    feedback_input = f"The team has collectively taken {total_leads} leads, generated ${total_revenue} in revenue, and booked {total_tours} tours. Provide a summary of the team's performance."
    team_feedback = model(feedback_input, max_length=100, num_return_sequences=1, truncation=True)[0]['generated_text']
    
    return jsonify({
        "total_leads": total_leads,
        "total_revenue": total_revenue,
        "total_tours": total_tours,
        "feedback": team_feedback
    })

# 3. Sales Performance Trends and Forecasting

@app.route('/api/performance_trends', methods=['GET'])
def performance_trends():
    time_period = request.args.get('time_period')
    data['dated'] = pd.to_datetime(data['dated'], errors='coerce') # Ensure 'dated' column is datetime
    
    # Analyze sales performance over the time period (e.g., monthly, quarterly)
    if time_period == 'monthly':
        period_data = data.groupby(data['dated'].dt.to_period('M')).sum(numeric_only=True)
    elif time_period == 'quarterly':
        period_data = data.groupby(data['dated'].dt.to_period('Q')).sum(numeric_only=True)
    else:
        return jsonify({"error": "Invalid time period. Use 'monthly' or 'quarterly'."}), 400

    # Generate trends and forecast feedback
    leads_trend = period_data['lead_taken'].mean()
    revenue_trend = period_data['revenue_confirmed'].mean()
    forecast_input = f"Based on the {time_period} data, the average leads are {leads_trend} and the average revenue is $ {revenue_trend}. Forecast the future performance."
    forecast = model(forecast_input, max_length=100, num_return_sequences=1)[0]['generated_text']

    return jsonify({
        "time_period": time_period,
        "average_leads": leads_trend,
        "average_revenue": revenue_trend,
        "forecast": forecast
    })
    


if __name__ == '__main__':
    app.run(port=3500)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:3500
Press CTRL+C to quit
